# ML-As-5

## Q-1

In [2]:
import numpy as np
from sklearn.cluster import KMeans

# Define the data matrix
X = np.array([[0, 0, 1, 5, 5],
              [2, 0, 0, 0, 2]]).T  # Transpose to have samples as rows

# Perform k-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Get the final cluster assignments and centroids
cluster_assignments = kmeans.labels_
centroids = kmeans.cluster_centers_

cluster_assignments, centroids

/opt/homebrew/Caskroom/miniforge/base/envs/BDA/lib/python3.10/site-packages/joblib/externals/loky/backend/context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[Errno 2] No such file or directory: 'sysctl'
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/BDA/lib/python3.10/site-packages/joblib/externals/loky/backend/context.py", line 270, in _count_physical_cores
    cpu_info = subprocess.run(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/BDA/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/opt/homebrew/Caskroom/miniforge/base/envs/BDA/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/opt/homebrew/Caskroom/miniforge/base/envs/BDA/lib/python3.1

(array([0, 0, 0, 1, 1], dtype=int32),
 array([[0.33333333, 0.66666667],
        [5.        , 1.        ]]))

## Q-2

In [3]:
import numpy as np

# Initial parameters
pi_0, p_0, q_0 = 0.46, 0.55, 0.67
observations = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1])
n = len(observations)

# EM algorithm
def expectation_maximization(pi, p, q, observations, iterations=100, tol=1e-6):
    for _ in range(iterations):
        # E-step: calculate responsibilities (probabilities for latent variables)
        responsibility_B = pi * (p ** observations) * ((1 - p) ** (1 - observations))
        responsibility_C = (1 - pi) * (q ** observations) * ((1 - q) ** (1 - observations))
        total_responsibility = responsibility_B + responsibility_C
        gamma_B = responsibility_B / total_responsibility
        gamma_C = responsibility_C / total_responsibility

        # M-step: update parameters
        pi_new = np.mean(gamma_B)
        p_new = np.sum(gamma_B * observations) / np.sum(gamma_B)
        q_new = np.sum(gamma_C * observations) / np.sum(gamma_C)

        # Check for convergence
        if abs(pi_new - pi) < tol and abs(p_new - p) < tol and abs(q_new - q) < tol:
            break

        pi, p, q = pi_new, p_new, q_new

    return pi, p, q

# Run the EM algorithm
pi_final, p_final, q_final = expectation_maximization(pi_0, p_0, q_0, observations)

pi_final, p_final, q_final

(0.461862835113919, 0.5345950037850112, 0.6561346417857326)

## Q-3

In [1]:
import numpy as np
from scipy.stats import norm

# Data
data = np.array([-67, -48, 6, 8, 14, 16, 23, 24, 28, 29, 41, 49, 56, 60, 75])

# Initialize parameters
np.random.seed(42)
alpha_0, alpha_1 = 0.5, 0.5
mu_0, mu_1 = np.random.choice(data, 2, replace=False)
sigma_0, sigma_1 = np.std(data), np.std(data)

# EM algorithm parameters
max_iter = 100
tolerance = 1e-4

# EM algorithm
for iteration in range(max_iter):
    # E-step: calculate responsibilities
    r0 = alpha_0 * norm.pdf(data, mu_0, sigma_0)
    r1 = alpha_1 * norm.pdf(data, mu_1, sigma_1)
    total_r = r0 + r1
    gamma_0 = r0 / total_r
    gamma_1 = r1 / total_r

    # M-step: update parameters
    new_alpha_0 = gamma_0.mean()
    new_alpha_1 = gamma_1.mean()
    new_mu_0 = np.sum(gamma_0 * data) / np.sum(gamma_0)
    new_mu_1 = np.sum(gamma_1 * data) / np.sum(gamma_1)
    new_sigma_0 = np.sqrt(np.sum(gamma_0 * (data - new_mu_0) ** 2) / np.sum(gamma_0))
    new_sigma_1 = np.sqrt(np.sum(gamma_1 * (data - new_mu_1) ** 2) / np.sum(gamma_1))

    # Convergence check
    if (
        np.abs(new_mu_0 - mu_0) < tolerance
        and np.abs(new_mu_1 - mu_1) < tolerance
        and np.abs(new_sigma_0 - sigma_0) < tolerance
        and np.abs(new_sigma_1 - sigma_1) < tolerance
    ):
        break

    # Update parameters
    alpha_0, alpha_1 = new_alpha_0, new_alpha_1
    mu_0, mu_1 = new_mu_0, new_mu_1
    sigma_0, sigma_1 = new_sigma_0, new_sigma_1

# Final parameters
alpha_0, mu_0, sigma_0, alpha_1, mu_1, sigma_1

(0.13317231632313895,
 -57.511074469836906,
 9.499993564737336,
 0.8668276836768611,
 32.9848906225897,
 20.723370648697156)